<a href="https://colab.research.google.com/github/DomThePom0411/ML-Tracker/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install

In [ ]:
!pip install roboflow
!pip install ultralytics
!pip install pyyaml
!pip install numpy  # Added this for the numpy arrays in the tracking code

Import

In [ ]:
from pathlib import Path
import torch
import os
import yaml
from roboflow import Roboflow
from ultralytics import YOLO
import numpy as np
from collections import defaultdict

Files + Augmentations

In [ ]:
class BasketballDetectionTrainer:
    def __init__(self,
                 roboflow_api_key,
                 base_path="/content/3x3-Player-Detection-Tracker-10",
                 model_size='s',
                 max_players=6):  # Removed checkpoint_path parameter
        self.api_key = roboflow_api_key
        self.base_path = Path(base_path)
        self.model_size = model_size
        self.max_players = max_players
        self.player_tracker = PlayerTracker(max_players=max_players)

        # CUDA setup
        torch.cuda.empty_cache()
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        if self.device == 'cuda':
            torch.cuda.set_device(0)
            print(f"Using CUDA device: {torch.cuda.get_device_name(0)}")
        print(f"Using device: {self.device}")

    def setup_dataset(self):
        """Download and setup the dataset from Roboflow."""
        print("Downloading dataset...")
        rf = Roboflow(api_key=self.api_key)
        project = rf.workspace("3x3-detection-base").project("3x3-player-detection-tracker")
        version = project.version(10)
        dataset = version.download("yolov8")
        print("Dataset download complete.")
        return dataset

    def create_yaml_config(self):
        """Create the YAML configuration with optimized settings for player detection."""
        yaml_content = {
            'path': str(self.base_path),
            'train': 'train/images',
            'val': 'valid/images',
            'test': 'test/images',
            'nc': 3,
            'names': ['ball', 'player', 'referee'],
            'augment': {
                'hsv_h': 0.015,
                'hsv_s': 0.7,
                'hsv_v': 0.4,
                'degrees': 0,
                'translate': 0.1,
                'scale': 0.3,
                'shear': 0.0,
                'perspective': 0.0,
                'flipud': 0.0,
                'fliplr': 0.5,
                'mosaic': 0.8,
                'mixup': 0.0,
                # Added augmentations for occlusion handling
                'copy_paste': 0.3,  # Copy-paste augmentation for better occlusion handling
                'blur': 0.1,        # Slight blur to help with motion
            }
        }

        yaml_path = self.base_path / 'data.yaml'
        with open(yaml_path, 'w') as file:
            yaml.dump(yaml_content, file, sort_keys=False)
        return yaml_path

    def verify_dataset(self):
        """Verify dataset structure and contents."""
        for dir_name in ['train/images', 'valid/images', 'test/images']:
            full_path = self.base_path / dir_name
            if full_path.exists():
                print(f"✓ {dir_name}: {len(list(full_path.glob('*')))} files")
            else:
                raise FileNotFoundError(f"Directory not found: {dir_name}")

    def train_model(self):
        """Train the model with optimized hyperparameters for player detection."""
        try:
            if self.device == 'cuda':
                torch.cuda.empty_cache()

            os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

            print("Loading fresh model...")
            model = YOLO(f'yolov8{self.model_size}.pt')
            print("Model loaded successfully.")

Training + IOU calculations

In [ ]:
  # Attemmpted to Modify training parameters for better occlusion handling - WORKED A LITTLE BIT, BUT STILL COUNTING TOO MANY PLAYERS AND STILL MERGING BOXES
            results = model.train(
                data=str(self.base_path / 'data.yaml'),
                epochs=150,
                imgsz=832,
                batch=2,
                nms=True,
                iou=0.7,  # Increased IoU threshold for better handling of overlapping players
                conf=0.4,  # Increased confidence threshold
                overlap_mask=True,  # Enable overlap handling
                max_det=self.max_players + 2,  # Max players plus some buffer
                verbose=True,
                patience=25,
                optimizer='SGD',
                lr0=0.01,
                lrf=0.01,
                momentum=0.937,
                weight_decay=0.0005,
                warmup_epochs=3,
                warmup_momentum=0.8,
                box=7.5,
                cls=0.3,
                dfl=1.5,
                close_mosaic=10,
                project='3x3_detection',
                name='train_optimized',
                exist_ok=True,
                save=True,
                cache=False,
                device=self.device,
                workers=0,
                resume=False  # Ensure we're not resuming from any checkpoint
            )
            return results

        except RuntimeError as e:
            if self.device == 'cuda':
                torch.cuda.empty_cache()
            raise e

    def post_process_detections(self, detections):
        """Post-process detections to handle multiple detections and occlusions."""
        # Convert detections to format: [x1, y1, x2, y2, conf, class]
        processed_detections = []

        # Apply custom NMS with temporal consistency
        filtered_detections = self.custom_nms(detections)

        # Update player tracker
        tracked_detections = self.player_tracker.update(filtered_detections)

        return tracked_detections

    def custom_nms(self, detections, iou_threshold=0.7, conf_threshold=0.4):
        """Custom NMS implementation for basketball players."""
        if len(detections) == 0:
            return []

        # Convert to numpy for easier manipulation
        boxes = np.array([d[:4] for d in detections])
        scores = np.array([d[4] for d in detections])

        # Filter by confidence
        mask = scores > conf_threshold
        boxes = boxes[mask]
        scores = scores[mask]

        # Calculate areas
        areas = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])

        # Sort by confidence
        order = scores.argsort()[::-1]

        keep = []
        while order.size > 0:
            i = order[0]
            keep.append(i)

            # Break if we've reached max players
            if len(keep) >= self.max_players:
                break

            if order.size == 1:
                break

            # Calculate IoU with remaining boxes
            xx1 = np.maximum(boxes[i, 0], boxes[order[1:], 0])
            yy1 = np.maximum(boxes[i, 1], boxes[order[1:], 1])
            xx2 = np.minimum(boxes[i, 2], boxes[order[1:], 2])
            yy2 = np.minimum(boxes[i, 3], boxes[order[1:], 3])

            w = np.maximum(0.0, xx2 - xx1)
            h = np.maximum(0.0, yy2 - yy1)
            inter = w * h

            ovr = inter / (areas[i] + areas[order[1:]] - inter)

            # Get indices of boxes with IoU <= threshold
            inds = np.where(ovr <= iou_threshold)[0]
            order = order[inds + 1]

        return [detections[i] for i in keep]

class PlayerTracker:
    """Track players across frames to maintain temporal consistency."""
    def __init__(self, max_players=6, history_length=5):
        self.max_players = max_players
        self.history_length = history_length
        self.tracked_players = defaultdict(list)
        self.next_id = 0

    def update(self, detections):
        """Update tracked players with new detections."""
        current_players = {}

        # Match detections to existing players
        for detection in detections:
            matched = False
            best_iou = 0
            best_id = None

            for player_id, history in self.tracked_players.items():
                if not history:
                    continue

                iou = self.calculate_iou(detection, history[-1])
                if iou > 0.3 and iou > best_iou:  # Lower threshold for matching
                    best_iou = iou
                    best_id = player_id
                    matched = True

            if matched:
                current_players[best_id] = detection
            elif len(current_players) < self.max_players:
                current_players[self.next_id] = detection
                self.next_id += 1

        # Update histories
        for player_id, detection in current_players.items():
            self.tracked_players[player_id].append(detection)
            if len(self.tracked_players[player_id]) > self.history_length:
                self.tracked_players[player_id].pop(0)

        # Remove old players
        current_ids = set(current_players.keys())
        self.tracked_players = {k: v for k, v in self.tracked_players.items()
                              if k in current_ids}

        return list(current_players.values())

    @staticmethod
    def calculate_iou(box1, box2):
        """Calculate IoU between two bounding boxes."""
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])

        intersection = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])

        return intersection / (area1 + area2 - intersection)

# Example usage
def main():
    API_KEY = # "INSERT API KEY HERE"

    # Initialize trainer with fresh training (no checkpoint)
    trainer = BasketballDetectionTrainer(
        roboflow_api_key=API_KEY,
        max_players=6  # Set maximum number of players - THIS DIDN'T REALLY WORK THAT WELL
    )

    try:
        print("Setting up dataset...")
        trainer.setup_dataset()

        print("Creating YAML configuration...")
        trainer.create_yaml_config()

        print("Verifying dataset structure...")
        trainer.verify_dataset()

        print("Starting fresh training...")
        results = trainer.train_model()

        print("Training completed successfully!")

    except Exception as e:
        print(f"Error occurred: {str(e)}")
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        raise

if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 11.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.8/914.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00

Extracting Dataset Version Zip to 3x3-Player-Detection-Tracker-10 in yolov8:: 100%|██████████| 2326/2326 [00:01<00:00, 2005.43it/s]


Dataset download complete.
Creating YAML configuration...
Verifying dataset structure...
✓ train/images: 845 files
✓ valid/images: 209 files
✓ test/images: 103 files
Starting fresh training...
Loading fresh model...


100%|██████████| 21.5M/21.5M [00:00<00:00, 215MB/s]


Model loaded successfully.
Ultralytics 8.3.72 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/3x3-Player-Detection-Tracker-10/data.yaml, epochs=150, time=None, patience=25, batch=2, imgsz=832, save=True, save_period=-1, cache=False, device=cuda, workers=0, project=3x3_detection, name=train_optimized, exist_ok=True, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.4, iou=0.7, max_det=8, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=Fals

...

                   all        209       1624      0.978      0.869      0.929      0.787



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    142/150     0.988G      0.391     0.1463       0.81          8        832: 100%|██████████| 423/423 [01:40<00:00,  4.20it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 53/53 [00:07<00:00,  7.47it/s]


                   all        209       1624      0.971      0.881      0.934      0.793

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    143/150      1.01G     0.3743     0.1406     0.8063         14        832:  31%|███       | 131/423 [00:31<01:06,  4.40it/s]